In [1]:
import math
import torch
import time
import random
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.parameter import Parameter
from torch.nn import init
from torch import Tensor
from scipy.special import gamma 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath, amssymb}'
custom_params = {
    # Font and text settings
    'font.size': 8,                
    'axes.labelsize': 8,           
    'axes.titlesize': 8,           
    'xtick.labelsize': 8,          
    'ytick.labelsize': 8,          
    'legend.fontsize': 6,          
    
    # Line properties
    'lines.linewidth': 1.2,        
    'lines.markersize': 4,         
    
    # Figure dimensions and quality
    'figure.figsize': (3.5, 2.5),  
    
    # Axis properties
    'axes.linewidth': 0.8,         
    'grid.linewidth': 0.5,         
    'grid.alpha': 0.3,             
}

# Apply the customized parameters to matplotlib
plt.rcParams.update(custom_params)


def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.666,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test

def MSE(pred,true):
    return np.mean((pred-true)**2)

def MAE(pred, true):
    return np.mean(np.abs(pred-true))

def RMSE(pred,true):
    return np.sqrt(np.mean((pred-true)**2))

def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def clip_matrix_norm(matrix, max_norm):
    norm = torch.norm(matrix)
    if norm > max_norm:
        matrix = matrix * (max_norm / norm)
    return matrix


class Fractional_Order_Matrix_Differential_Solver(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input1,w,b,alpha,c):
        alpha = torch.tensor(alpha)
        c = torch.tensor(c)
        ctx.save_for_backward(input1,w,b,alpha,c)
        outputs = input1@w + b
        return outputs

    @staticmethod
    def backward(ctx, grad_outputs):
        input1,w,b,alpha,c = ctx.saved_tensors
        x_fractional, w_fractional = Fractional_Order_Matrix_Differential_Solver.Fractional_Order_Matrix_Differential_Linear(input1,w,b,alpha,c)   
        x_grad = grad_outputs@x_fractional
        w_grad = w_fractional@grad_outputs
        b_grad = grad_outputs.sum(dim=0)
        return x_grad, w_grad, b_grad,None,None
          
    @staticmethod
    def Fractional_Order_Matrix_Differential_Linear(xs,ws,b,alpha,c):
        wf = ws[:,0].view(1,-1)
        #main
        w_main = torch.mul(xs,(torch.abs(wf)+1e-8)**(1-alpha)/gamma(2-alpha))
        #partial
        w_partial = torch.mul((xs@wf.T).expand(xs.shape) - torch.mul(xs,wf) + b[0], torch.sgn(wf)*(torch.abs(wf)+1e-8)**(-alpha)/gamma(1-alpha))
        return ws.T, (w_main + clip_matrix_norm(w_partial,c)).transpose(-2,-1)

class FLinear(nn.Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, alpha=0.9, c=1.0, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.c = c
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        return Fractional_Order_Matrix_Differential_Solver.apply(x, self.weight.T, self.bias, self.alpha,self.c)

    def extra_repr(self) -> str:
        return f"in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None}"

In [2]:
from torch.optim import Optimizer

class Lookahead:
    def __init__(self, optimizer, alpha=0.5, k=6):
        if not 0.0 < alpha <= 1.0:
            raise ValueError(f"Invalid alpha: {alpha}")
        if k < 1:
            raise ValueError(f"Invalid k: {k}")

        self.optimizer = optimizer
        self.alpha = alpha
        self.k = k
        self.step_counter = 0

        self.slow_weights = [
            p.detach().clone()
            for group in optimizer.param_groups
            for p in group['params']
        ]
        for w in self.slow_weights:
            w.requires_grad = False

    def zero_grad(self):
        self.optimizer.zero_grad()

    def step(self, closure=None):
        loss = self.optimizer.step(closure)
        self.step_counter += 1

        if self.step_counter % self.k == 0:
            idx = 0
            for group in self.optimizer.param_groups:
                for p in group['params']:
                    slow = self.slow_weights[idx]
                    slow.data.add_(p.data - slow.data, alpha=self.alpha)
                    p.data.copy_(slow.data)
                    idx += 1
        return loss


In [3]:
# Configuration parameters
slide_windows_size = 192  # Input sequence length
pred_length = 384        # Prediction horizon length
stock = 'ETTh1'            # Dataset name (DJI for comparison)
features_j = 6           # Target feature index (DJI:4, ETTm2:6)
num_feature = features_j + 1         #(DJI:5, ETTm2:7)

# Load data
root = r'C:\Users\Administrator\torch_zxj\博士第四篇代码'
df_DJIA = pd.read_csv(root+'/data/'+stock+'.csv')
# df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
del df_DJIA['date']  # Remove date column

# 1. Split data first (7:1:2 ratio)
def split_time_series(data, train_ratio=0.7, val_ratio=0.1):
    """
    Split time series data in chronological order
    
    Args:
        data: Complete time series data
        train_ratio: Proportion for training set
        val_ratio: Proportion for validation set
    
    Returns:
        train_data, val_data, test_data: Split datasets
    """
    n_samples = len(data)
    train_end = int(n_samples * train_ratio)
    val_end = train_end + int(n_samples * val_ratio)
    
    # Split in chronological order (important for time series)
    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]
    
    return train_data, val_data, test_data

# Split raw data first
train_raw, val_raw, test_raw = split_time_series(df_DJIA.values, 0.7, 0.1)

print(f"Data split results:")
print(f"  Training set: {len(train_raw)} samples ({len(train_raw)/len(df_DJIA)*100:.1f}%)")
print(f"  Validation set: {len(val_raw)} samples ({len(val_raw)/len(df_DJIA)*100:.1f}%)")
print(f"  Test set: {len(test_raw)} samples ({len(test_raw)/len(df_DJIA)*100:.1f}%)")

# scaler = MinMaxScaler()
scaler = StandardScaler()
scaler.fit(train_raw)

# Transform all datasets using training set statistics
train_scaled = scaler.transform(train_raw)
val_scaled = scaler.transform(val_raw)  # Use training set statistics
test_scaled = scaler.transform(test_raw)  # Use training set statistics

# 3. Create sequences for time series forecasting
def create_sequences(data, slide_windows_size, pred_length, target_idx):
    """
    Create input-output sequences for time series forecasting
    
    Args:
        data: Multivariate time series data
        slide_windows_size: Input sequence length (look-back window)
        pred_length: Output sequence length (forecast horizon)
        target_idx: Index of target feature to predict
    
    Returns:
        X: Input sequences [samples, seq_len, features]
        y: Target sequences [samples, pred_length]
    """
    X, y = [], []
    for i in range(len(data) - slide_windows_size - pred_length + 1):
        # Input sequence: sliding window of features
        X.append(data[i:i+slide_windows_size, :])  # [seq_len, features]
        # Target sequence: future values of target feature
        y.append(data[i+slide_windows_size:i+slide_windows_size+pred_length, target_idx])  
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

# Create sequences for each dataset
X_train, y_train = create_sequences(train_scaled, slide_windows_size, pred_length, features_j)
X_val, y_val = create_sequences(val_scaled, slide_windows_size, pred_length, features_j)
X_test, y_test = create_sequences(test_scaled, slide_windows_size, pred_length, features_j)

print(f"\nSequence creation results:")
print(f"  Training set: X{X_train.shape}, y{y_train.shape}")
print(f"  Validation set: X{X_val.shape}, y{y_val.shape}")
print(f"  Test set: X{X_test.shape}, y{y_test.shape}")

# 4. Convert to PyTorch tensors
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

print(f"\nTensor shapes:")
print(f"  X_train_tensor: {X_train_tensor.shape}")
print(f"  y_train_tensor: {y_train_tensor.shape}")
print(f"  X_val_tensor: {X_val_tensor.shape}")
print(f"  y_val_tensor: {y_val_tensor.shape}")
print(f"  X_test_tensor: {X_test_tensor.shape}")
print(f"  y_test_tensor: {y_test_tensor.shape}")

Data split results:
  Training set: 12194 samples (70.0%)
  Validation set: 1742 samples (10.0%)
  Test set: 3484 samples (20.0%)

Sequence creation results:
  Training set: X(11619, 192, 7), y(11619, 384)
  Validation set: X(1167, 192, 7), y(1167, 384)
  Test set: X(2909, 192, 7), y(2909, 384)

Using device: cuda:0

Tensor shapes:
  X_train_tensor: torch.Size([11619, 192, 7])
  y_train_tensor: torch.Size([11619, 384])
  X_val_tensor: torch.Size([1167, 192, 7])
  y_val_tensor: torch.Size([1167, 384])
  X_test_tensor: torch.Size([2909, 192, 7])
  y_test_tensor: torch.Size([2909, 384])


In [4]:
batch_size = 256
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [ ]:
# momentums = [0.1,0.5,0.9]
weight_decays = [1e-3,1e-4,1e-5]
lrs = [3e-2,1e-2,3e-3,1e-3,3e-4]             

# root = r'C:\Users\Administrator\torch_zxj\博士第四篇代码'
# batch_size = 256
num_epochs = 200
set_seed()
# train_data = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1=512, hidden_size2=256,output_size=pred_length):  
        super().__init__()
        self.flatten = nn.Flatten()
        # self.linear1 = FLinear(input_size, hidden_size1, alpha,c) 
        self.linear1 = nn.Linear(input_size, hidden_size1)    
        self.leakrelu1 = nn.LeakyReLU()                          
        # self.linear2 = FLinear(hidden_size1, hidden_size2, alpha,c) 
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)    
        self.leakrelu2 = nn.LeakyReLU()
        # self.linear3 = FLinear(hidden_size2, output_size, alpha,c)  
        self.linear3 = nn.Linear(hidden_size2, output_size)  
        
    def forward(self, x):
        x = self.flatten(x)    # (batch_size, seq_len*num_features)
        x = self.leakrelu1(self.linear1(x))
        x = self.leakrelu2(self.linear2(x))
        x = self.linear3(x)
        return x
best_lr = 0
best_momentum = 0
best_weight_decay = 0
best_evaluation = 10000
for lr in lrs:
    for weight_decay in weight_decays:   
        print('')
        print('lr:',lr)
        print('weight_decay:',weight_decay)
        set_seed()
        model = MLP(input_size=slide_windows_size*num_feature).to(device)
        best_loss = 10000
        criterion = nn.MSELoss()
        optimizer_base = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)   #adam
        optimizer = Lookahead(optimizer_base)
        for ii in range(num_epochs):
            model.train()
            loss_sum = 0
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss_sum += loss
                loss.backward()   #The default value of retain_graph is False.
                optimizer.step()
                
            model.eval()
            with torch.no_grad():
                Val_outputs = model(X_val_tensor)
                RMSE_val = RMSE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAE_val = MAE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAPE_val = MAPE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                best_val = RMSE_val + MAE_val+MAPE_val

                if best_loss > best_val:
                    best_loss = best_val
                    torch.save(model.state_dict(), root+'/model/table4/'+stock+'_model_fractional'+'_'+str(weight_decay)+'_'+str(lr)+'_.pth')    #adam

        model.load_state_dict(torch.load(root+'/model/table4/'+stock+'_model_fractional'+'_'+str(weight_decay)+'_'+str(lr)+'_.pth'))   #adam

        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
        RMSE_test = RMSE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAE_test = MAE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAPE_test = MAPE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        print(str(lr)+'_'+str(weight_decay)+'_'+f'RMSE:{RMSE_test:.6f}')
        print(str(lr)+'_'+str(weight_decay)+'_'+f'MAE:{MAE_test:.6f}')
        print(str(lr)+'_'+str(weight_decay)+'_'+f'MAPE:{MAPE_test:.6f}')
        print('RMSE_test+MAE_test+MAPE_test:',RMSE_test+MAE_test+MAPE_test)
        if best_evaluation > RMSE_test + MAE_test + MAPE_test:
            best_evaluation = RMSE_test + MAE_test + MAPE_test
            print('Best evaluation:',best_evaluation)
            best_lr = lr
            best_weight_decay = weight_decay
print('best_lr:',best_lr)
print('best_weight_decay:',best_weight_decay)
print('best_evaluation:',best_evaluation)


lr: 0.03
momentum: 0.1
weight_decay: 0.001
0.03_0.001_RMSE:0.428687
0.03_0.001_MAE:0.346937
0.03_0.001_MAPE:0.577869
RMSE_test+MAE_test+MAPE_test: 1.3534937
Best evaluation: 1.3534937

lr: 0.03
momentum: 0.1
weight_decay: 0.0001
0.03_0.0001_RMSE:0.427363
0.03_0.0001_MAE:0.345656
0.03_0.0001_MAPE:0.575970
RMSE_test+MAE_test+MAPE_test: 1.3489892
Best evaluation: 1.3489892

lr: 0.03
momentum: 0.1
weight_decay: 1e-05
0.03_1e-05_RMSE:0.427240
0.03_1e-05_MAE:0.345538
0.03_1e-05_MAPE:0.575726
RMSE_test+MAE_test+MAPE_test: 1.3485044
Best evaluation: 1.3485044

lr: 0.03
momentum: 0.5
weight_decay: 0.001
0.03_0.001_RMSE:0.428745
0.03_0.001_MAE:0.347018
0.03_0.001_MAPE:0.580560
RMSE_test+MAE_test+MAPE_test: 1.356322

lr: 0.03
momentum: 0.5
weight_decay: 0.0001
0.03_0.0001_RMSE:0.427325
0.03_0.0001_MAE:0.345659
0.03_0.0001_MAPE:0.575907
RMSE_test+MAE_test+MAPE_test: 1.3488907

lr: 0.03
momentum: 0.5
weight_decay: 1e-05
0.03_1e-05_RMSE:0.427191
0.03_1e-05_MAE:0.345530
0.03_1e-05_MAPE:0.575500
RMSE

C:\Users\Administrator\AppData\Local\Temp\ipykernel_42396\2194696802.py:64: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((pred - true) / true))


0.03_0.001_RMSE:0.421080
0.03_0.001_MAE:0.340253
0.03_0.001_MAPE:0.562912
RMSE_test+MAE_test+MAPE_test: 1.3242452
Best evaluation: 1.3242452

lr: 0.03
momentum: 0.9
weight_decay: 0.0001
0.03_0.0001_RMSE:0.419905
0.03_0.0001_MAE:0.339131
0.03_0.0001_MAPE:0.559616
RMSE_test+MAE_test+MAPE_test: 1.3186522
Best evaluation: 1.3186522

lr: 0.03
momentum: 0.9
weight_decay: 1e-05
0.03_1e-05_RMSE:0.419787
0.03_1e-05_MAE:0.339019
0.03_1e-05_MAPE:0.559366
RMSE_test+MAE_test+MAPE_test: 1.3181723
Best evaluation: 1.3181723

lr: 0.01
momentum: 0.1
weight_decay: 0.001
0.01_0.001_RMSE:0.429399
0.01_0.001_MAE:0.347558
0.01_0.001_MAPE:0.581684
RMSE_test+MAE_test+MAPE_test: 1.3586414

lr: 0.01
momentum: 0.1
weight_decay: 0.0001
0.01_0.0001_RMSE:0.428515
0.01_0.0001_MAE:0.346681
0.01_0.0001_MAPE:0.580847
RMSE_test+MAE_test+MAPE_test: 1.3560426

lr: 0.01
momentum: 0.1
weight_decay: 1e-05
0.01_1e-05_RMSE:0.428381
0.01_1e-05_MAE:0.346552
0.01_1e-05_MAPE:0.580245
RMSE_test+MAE_test+MAPE_test: 1.3551781

lr: 0.

In [ ]:
weight_decay = 1e-5
lr = 0.03

alphas = [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2]    # DJI:0.999,0.99,0.98,0.95,0.9,0.85,0.8,0.75  ETTh1:0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2
cs = [0.1,0.2,0.5,1.0,2.0,5.0,10.0]

# root = r'C:\Users\Administrator\torch_zxj\博士第四篇代码'
# batch_size = 256
num_epochs = 200
set_seed()
# train_data = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1=512, hidden_size2=256,output_size=pred_length):  
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = FLinear(input_size, hidden_size1, alpha,c) 
        # self.linear1 = nn.Linear(input_size, hidden_size1)    
        self.leakrelu1 = nn.LeakyReLU()                          
        self.linear2 = FLinear(hidden_size1, hidden_size2, alpha,c) 
        # self.linear2 = nn.Linear(hidden_size1, hidden_size2)    
        self.leakrelu2 = nn.LeakyReLU()
        self.linear3 = FLinear(hidden_size2, output_size, alpha,c)  
        # self.linear3 = nn.Linear(hidden_size2, output_size)  
        
    def forward(self, x):
        x = self.flatten(x)    # (batch_size, seq_len*num_features)
        x = self.leakrelu1(self.linear1(x))
        x = self.leakrelu2(self.linear2(x))
        x = self.linear3(x)
        return x
best_c = 0
best_alpha = 0
best_evaluation = 10000
for alpha in alphas:
    for c in cs:
        print('')
        print('alpha:',alpha)
        print('c:',c)
        set_seed()
        model = MLP(input_size=slide_windows_size*num_feature).to(device)
        best_loss = 10000
        criterion = nn.MSELoss()
        # optimizer_base = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)   #adam
        optimizer_base = torch.optim.SGD(model.parameters(), lr=lr,weight_decay=weight_decay,momentum=momentum)   #sgd
        optimizer = Lookahead(optimizer_base)
        for ii in range(num_epochs):
            model.train()
            loss_sum = 0
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss_sum += loss
                loss.backward()   #The default value of retain_graph is False.
                optimizer.step()
                
            model.eval()
            with torch.no_grad():
                Val_outputs = model(X_val_tensor)
                RMSE_val = RMSE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAE_val = MAE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAPE_val = MAPE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                best_val = RMSE_val + MAE_val+MAPE_val

                if best_loss > best_val:
                    best_loss = best_val
                    torch.save(model.state_dict(), root+'/model/table4/'+stock+'_model_fractional_adam'+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth')  #Fadam

        model.load_state_dict(torch.load(root+'/model/table4/'+stock+'_model_fractional_adam'+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth'))   #Fadam
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
        RMSE_test = RMSE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAE_test = MAE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAPE_test = MAPE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        print(f'RMSE:{RMSE_test:.6f}')
        print(f'MAE:{MAE_test:.6f}')
        print(f'MAPE:{MAPE_test:.6f}')
        print(f'RMSE+MAE+MAPE:{RMSE_test+MAE_test+MAPE_test:.6f}')
        if best_evaluation > RMSE_test + MAE_test + MAPE_test:
            best_evaluation = RMSE_test + MAE_test + MAPE_test
            print('Best evaluation:',best_evaluation)
            best_alpha = alpha
            best_c = c
print('best_alpha:',best_alpha)
print('best_c:',best_c)
print('best_evaluation:',best_evaluation)



alpha: 1.0
c: 0.1
RMSE:0.419787
MAE:0.339019
MAPE:0.559364
RMSE+MAE+MAPE:1.318171
Best evaluation: 1.3181708

alpha: 1.0
c: 0.2
RMSE:0.419787
MAE:0.339019
MAPE:0.559364
RMSE+MAE+MAPE:1.318171

alpha: 1.0
c: 0.5
RMSE:0.419787
MAE:0.339019
MAPE:0.559364
RMSE+MAE+MAPE:1.318171

alpha: 1.0
c: 1.0
RMSE:0.419787
MAE:0.339019
MAPE:0.559364
RMSE+MAE+MAPE:1.318171

alpha: 1.0
c: 2.0
RMSE:0.419787
MAE:0.339019
MAPE:0.559364
RMSE+MAE+MAPE:1.318171

alpha: 1.0
c: 5.0
RMSE:0.419787
MAE:0.339019
MAPE:0.559364
RMSE+MAE+MAPE:1.318171

alpha: 1.0
c: 10.0
RMSE:0.419787
MAE:0.339019
MAPE:0.559364
RMSE+MAE+MAPE:1.318171

alpha: 0.9
c: 0.1
RMSE:0.424161
MAE:0.342879
MAPE:0.567814
RMSE+MAE+MAPE:1.334854

alpha: 0.9
c: 0.2
RMSE:0.424168
MAE:0.342884
MAPE:0.567865
RMSE+MAE+MAPE:1.334917

alpha: 0.9
c: 0.5
RMSE:0.424162
MAE:0.342878
MAPE:0.567817
RMSE+MAE+MAPE:1.334857

alpha: 0.9
c: 1.0
RMSE:0.424147
MAE:0.342861
MAPE:0.567845
RMSE+MAE+MAPE:1.334852

alpha: 0.9
c: 2.0
RMSE:0.424164
MAE:0.342870
MAPE:0.568287

C:\Users\Administrator\AppData\Local\Temp\ipykernel_42396\2194696802.py:64: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((pred - true) / true))


RMSE:0.422435
MAE:0.340502
MAPE:0.525353
RMSE+MAE+MAPE:1.288290

alpha: 0.4
c: 5.0
RMSE:0.419156
MAE:0.337647
MAPE:0.516229
RMSE+MAE+MAPE:1.273032
Best evaluation: 1.2730322

alpha: 0.4
c: 10.0
RMSE:0.416718
MAE:0.335355
MAPE:0.509216
RMSE+MAE+MAPE:1.261288
Best evaluation: 1.2612884

alpha: 0.3
c: 0.1
RMSE:0.422334
MAE:0.340023
MAPE:0.515725
RMSE+MAE+MAPE:1.278082

alpha: 0.3
c: 0.2
RMSE:0.422215
MAE:0.339910
MAPE:0.515332
RMSE+MAE+MAPE:1.277457

alpha: 0.3
c: 0.5
RMSE:0.421835
MAE:0.339554
MAPE:0.514042
RMSE+MAE+MAPE:1.275432

alpha: 0.3
c: 1.0
RMSE:0.421427
MAE:0.339213
MAPE:0.513466
RMSE+MAE+MAPE:1.274106

alpha: 0.3
c: 2.0
RMSE:0.419874
MAE:0.337823
MAPE:0.508903
RMSE+MAE+MAPE:1.266600

alpha: 0.3
c: 5.0
RMSE:0.417231
MAE:0.335524
MAPE:0.502769
RMSE+MAE+MAPE:1.255524
Best evaluation: 1.2555237

alpha: 0.3
c: 10.0
RMSE:0.416853
MAE:0.334797
MAPE:0.498591
RMSE+MAE+MAPE:1.250242
Best evaluation: 1.2502415

alpha: 0.2
c: 0.1
RMSE:0.421318
MAE:0.338499
MAPE:0.502591
RMSE+MAE+MAPE:1.262